<a href="https://colab.research.google.com/github/sada1908/githublearnings/blob/main/cats_and_dogs_snr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import zipfile

In [11]:
!pip install kaggle

In [12]:
!mkdir -p ~/.kaggle



In [13]:
!mv /kaggle.json ~/.kaggle

In [14]:
!chmod 200 ~/.kaggle/kaggle.json

In [15]:
!kaggle datasets list

ref                                                             title                                               size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  -------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
amanbarthwal/steam-store-data                                   Steam Store Data                                     6MB  2024-05-25 06:23:06            958         29  1.0              
khushikhushikhushi/imdb-top-rated-tv-series-dataset             IMDb Top-Rated TV Series Dataset                    73KB  2024-05-25 19:22:33            603         22  1.0              
kanchana1990/vehicle-dataset-2024                               Vehicle Dataset 2024                               308KB  2024-05-29 20:02:30           1052         34  1.0              
shreyanshverma27/online-sales-dataset-popular-marketplace-data  O

In [16]:
!kaggle datasets download -d tongpython/cat-and-dog

Dataset URL: https://www.kaggle.com/datasets/tongpython/cat-and-dog
License(s): CC0-1.0
 91% 199M/218M [00:02<00:00, 53.0MB/s]
100% 218M/218M [00:02<00:00, 78.3MB/s]


In [17]:
os.makedirs('/content1/cats_and_dogs',exist_ok=True)

In [18]:
with zipfile.ZipFile('/content/cat-and-dog.zip','r') as zip_ref:
  zip_ref.extractall('/content1/cats_and_dogs')

In [19]:
test_cats_set = os.listdir('/content1/cats_and_dogs/test_set/test_set/cats')
test_dogs_set = os.listdir('/content1/cats_and_dogs/test_set/test_set/dogs')
train_cats_set = os.listdir('/content1/cats_and_dogs/training_set/training_set/cats')
train_dogs_set = os.listdir('/content1/cats_and_dogs/training_set/training_set/dogs')

In [20]:
len(train_cats_set)

4001

In [21]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
import os
import numpy as np
import matplotlib.pyplot as plt


In [22]:
base_model = VGG16(weights='imagenet',include_top=False,input_shape=(224,224,3))

58889256/58889256 [==============================] - 0s 0us/step


In [23]:
base_model.trainable = False

In [24]:
x = base_model.output
x = Flatten()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1,activation='sigmoid')(x)
predictions = Dense(1,activation='sigmoid')(x)
model = Model(inputs=base_model.input,outputs=predictions)


In [25]:
new_test_dir = '/content1/cats_and_dogs/test_set/test_set'
new_train_dir = '/content1/cats_and_dogs/training_set/training_set'

In [26]:
train_datagen =ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,

)

In [27]:
validation_datagen = ImageDataGenerator(
    rescale=1./255
)

In [28]:
train_generator = train_datagen.flow_from_directory(
    new_train_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 8005 images belonging to 2 classes.


In [29]:
validation_generator = validation_datagen.flow_from_directory(
    new_test_dir,
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 2023 images belonging to 2 classes.


In [30]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='binary_crossentropy',metrics=['accuracy'])

In [31]:
history = model.fit(train_generator,epochs=10,validation_data=validation_generator,
                    steps_per_epoch=len(train_generator),validation_steps=len(validation_generator))

Epoch 1/10
251/251 [==============================] - 129s 468ms/step - loss: 0.5843 - accuracy: 0.6603 - val_loss: 0.5292 - val_accuracy: 0.8087
Epoch 2/10
251/251 [==============================] - 116s 461ms/step - loss: 0.5306 - accuracy: 0.8322 - val_loss: 0.5235 - val_accuracy: 0.8166
Epoch 3/10
251/251 [==============================] - 117s 465ms/step - loss: 0.5163 - accuracy: 0.8703 - val_loss: 0.5037 - val_accuracy: 0.9090
Epoch 4/10
251/251 [==============================] - 117s 467ms/step - loss: 0.5062 - accuracy: 0.8959 - val_loss: 0.4977 - val_accuracy: 0.9199
Epoch 5/10
251/251 [==============================] - 116s 462ms/step - loss: 0.4973 - accuracy: 0.9053 - val_loss: 0.4887 - val_accuracy: 0.9199
Epoch 6/10
251/251 [==============================] - 118s 471ms/step - loss: 0.4899 - accuracy: 0.9126 - val_loss: 0.4846 - val_accuracy: 0.9174
Epoch 7/10
251/251 [==============================] - 116s 460ms/step - loss: 0.4813 - accuracy: 0.9218 - val_loss: 0.4774 -

In [32]:
val_loss,val_acc = model.evaluate(validation_generator,steps=len(validation_generator))
print(val_loss,val_acc)

64/64 [==============================] - 9s 140ms/step - loss: 0.4614 - accuracy: 0.9278
0.461353600025177 0.9278299808502197


In [33]:
model.save('cats_and_dogs.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [34]:
from tensorflow.keras.models import load_model

In [35]:
model1 = load_model('cats_and_dogs.h5')

In [36]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [37]:
def prepare_image(file):
    img_path = '/'
    img = load_img(img_path + file, target_size=(224, 224))  # Load image with target size
    img_array = img_to_array(img)  # Convert image to array
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to [0, 1]
    return img_array

In [39]:
# Predict a single image
image_file = 'pexels-pixabay-104827_1.jpg'
image = prepare_image(image_file)
prediction = model.predict(image)

# Convert prediction to class label
predicted_class = "Dog" if prediction > 0.5 else "Cat"

print(f'The predicted class for {image_file} is: {predicted_class}')

1/1 [==============================] - 0s 19ms/step
The predicted class for pexels-pixabay-104827_1.jpg is: Cat
